In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain


In [153]:
train1 = pd.read_csv("train/train9.csv")
train9 =pd.read_csv("train/train1.csv")
hero=pd.read_csv("train/hero_data.csv")
hero_win=pd.read_csv("hero_info.csv")
user_win=pd.read_csv("user_info.csv")

In [171]:
test1 = pd.read_csv("test/test1.csv")
test9 = pd.read_csv("test/test9.csv")

In [155]:
sub = pd.read_csv("test/test1.csv")

In [156]:
all_roles = hero['roles']
all_roles = np.unique(list(chain.from_iterable([i.split(':') for i in all_roles])))
def role_present(x, role):
    if x.find(role)==-1:
        return 0
    return 1

for role in all_roles:
    hero[role] = hero['roles'].apply(lambda x: role_present(x,role))

hero = hero.drop(['roles'],axis=1)

In [157]:
hero_win.columns

Index(['hero_id', 'Sum of num_wins', 'Sum of num_games',
       'win_hero_percentage'],
      dtype='object')

In [158]:
final_train_data = [train1, train9, test9]
final_train_data = pd.concat(final_train_data)
train = pd.merge(final_train_data, hero, on='hero_id', how='left')
train = pd.merge(train,hero_win[['hero_id','win_hero_percentage']],on='hero_id',how='left')
train = pd.merge(train,user_win[['user_id','win_user_per']],on='user_id',how='left')

In [159]:
for col in train.columns:
    print (col,' : ', train[col].dtype)

user_id  :  int64
hero_id  :  int64
id  :  object
num_games  :  int64
num_wins  :  int64
kda_ratio  :  float64
primary_attr  :  object
attack_type  :  object
base_health  :  int64
base_health_regen  :  float64
base_mana  :  int64
base_mana_regen  :  float64
base_armor  :  float64
base_magic_resistance  :  int64
base_attack_min  :  int64
base_attack_max  :  int64
base_strength  :  int64
base_agility  :  int64
base_intelligence  :  int64
strength_gain  :  float64
agility_gain  :  float64
intelligence_gain  :  float64
attack_range  :  int64
projectile_speed  :  int64
attack_rate  :  float64
move_speed  :  int64
turn_rate  :  float64
Carry  :  int64
Disabler  :  int64
Durable  :  int64
Escape  :  int64
Initiator  :  int64
Jungler  :  int64
Nuker  :  int64
Pusher  :  int64
Support  :  int64
win_hero_percentage  :  float64
win_user_per  :  float64


In [160]:
cat_features = list(all_roles) + ['primary_attr','attack_type','base_magic_resistance']
all_primary_attr = dict([(j,i) for i,j in enumerate(np.unique(train['primary_attr']))])
all_attack_type = dict([(j,i) for i,j in enumerate(np.unique(train['attack_type']))])

In [161]:
target = train['kda_ratio']
train = train.drop(['user_id','hero_id','id', 'kda_ratio','num_wins','base_health','base_mana','base_mana_regen'],axis=1)
train['primary_attr'] = train['primary_attr'].apply(lambda x: all_primary_attr[x])
train['attack_type'] = train['attack_type'].apply(lambda x: all_attack_type[x])

In [173]:
test = pd.merge(test1, hero, on='hero_id', how='left')
test = pd.merge(test,hero_win[['hero_id','win_hero_percentage']],on='hero_id',how='left')
test = pd.merge(test,user_win[['user_id','win_user_per']],on='user_id',how='left')
all_primary_attr = dict([(j,i) for i,j in enumerate(np.unique(test['primary_attr']))])
all_attack_type = dict([(j,i) for i,j in enumerate(np.unique(test['attack_type']))])
test = test.drop(['user_id','hero_id','id','base_health','base_mana','base_mana_regen'],axis=1)
test['primary_attr'] = test['primary_attr'].apply(lambda x: all_primary_attr[x])
test['attack_type'] = test['attack_type'].apply(lambda x: all_attack_type[x])

In [174]:
for col in train.columns:
    print (col,' : ', train[col].dtype)

num_games  :  int64
primary_attr  :  int64
attack_type  :  int64
base_health_regen  :  float64
base_armor  :  float64
base_magic_resistance  :  int64
base_attack_min  :  int64
base_attack_max  :  int64
base_strength  :  int64
base_agility  :  int64
base_intelligence  :  int64
strength_gain  :  float64
agility_gain  :  float64
intelligence_gain  :  float64
attack_range  :  int64
projectile_speed  :  int64
attack_rate  :  float64
move_speed  :  int64
turn_rate  :  float64
Carry  :  int64
Disabler  :  int64
Durable  :  int64
Escape  :  int64
Initiator  :  int64
Jungler  :  int64
Nuker  :  int64
Pusher  :  int64
Support  :  int64
win_hero_percentage  :  float64
win_user_per  :  float64


In [164]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

In [165]:
cat_features_idx = [i for i,j in enumerate(list(train.columns)) if j in cat_features]
cat_features_idx

[1, 2, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [166]:
cb_model_with_features = CatBoostRegressor(iterations=4000, learning_rate=0.01, depth=6)

In [167]:
cb_model_with_features.fit(train, target, cat_features_idx)

0:	learn: 3649.9912406	total: 12.4ms	remaining: 49.7s
1:	learn: 3615.9564757	total: 27.7ms	remaining: 55.4s
2:	learn: 3582.4272879	total: 43.8ms	remaining: 58.3s
3:	learn: 3549.4216092	total: 54.1ms	remaining: 54s
4:	learn: 3516.6734064	total: 66.3ms	remaining: 53s
5:	learn: 3484.3939402	total: 87.4ms	remaining: 58.2s
6:	learn: 3452.3641795	total: 110ms	remaining: 1m 2s
7:	learn: 3420.5513051	total: 126ms	remaining: 1m 3s
8:	learn: 3389.1195599	total: 137ms	remaining: 1m
9:	learn: 3358.0770260	total: 152ms	remaining: 1m
10:	learn: 3327.4387307	total: 162ms	remaining: 58.9s
11:	learn: 3297.0076918	total: 174ms	remaining: 57.7s
12:	learn: 3266.9331029	total: 188ms	remaining: 57.8s
13:	learn: 3237.0795032	total: 203ms	remaining: 57.8s
14:	learn: 3207.7228276	total: 221ms	remaining: 58.6s
15:	learn: 3178.4729893	total: 241ms	remaining: 60s
16:	learn: 3149.6842703	total: 260ms	remaining: 1m
17:	learn: 3120.9780109	total: 279ms	remaining: 1m 1s
18:	learn: 3093.1058463	total: 291ms	remaining:

160:	learn: 1127.3468373	total: 2.95s	remaining: 1m 10s
161:	learn: 1122.5095754	total: 2.98s	remaining: 1m 10s
162:	learn: 1117.6811216	total: 3.01s	remaining: 1m 10s
163:	learn: 1112.9596018	total: 3.02s	remaining: 1m 10s
164:	learn: 1108.3041395	total: 3.05s	remaining: 1m 10s
165:	learn: 1103.6298376	total: 3.07s	remaining: 1m 10s
166:	learn: 1099.2367309	total: 3.09s	remaining: 1m 10s
167:	learn: 1094.8625415	total: 3.11s	remaining: 1m 10s
168:	learn: 1090.5047829	total: 3.12s	remaining: 1m 10s
169:	learn: 1086.1006708	total: 3.14s	remaining: 1m 10s
170:	learn: 1081.8084953	total: 3.16s	remaining: 1m 10s
171:	learn: 1077.7344792	total: 3.18s	remaining: 1m 10s
172:	learn: 1073.8528715	total: 3.19s	remaining: 1m 10s
173:	learn: 1069.9038358	total: 3.21s	remaining: 1m 10s
174:	learn: 1065.7291521	total: 3.23s	remaining: 1m 10s
175:	learn: 1061.7975467	total: 3.25s	remaining: 1m 10s
176:	learn: 1057.9244432	total: 3.26s	remaining: 1m 10s
177:	learn: 1054.0583780	total: 3.28s	remaining:

318:	learn: 832.7509991	total: 5.69s	remaining: 1m 5s
319:	learn: 832.2201113	total: 5.71s	remaining: 1m 5s
320:	learn: 831.8208409	total: 5.72s	remaining: 1m 5s
321:	learn: 831.2754262	total: 5.74s	remaining: 1m 5s
322:	learn: 830.6962823	total: 5.76s	remaining: 1m 5s
323:	learn: 830.2949113	total: 5.77s	remaining: 1m 5s
324:	learn: 829.8058372	total: 5.79s	remaining: 1m 5s
325:	learn: 829.4225876	total: 5.8s	remaining: 1m 5s
326:	learn: 828.8177252	total: 5.81s	remaining: 1m 5s
327:	learn: 828.2064426	total: 5.83s	remaining: 1m 5s
328:	learn: 827.8502062	total: 5.84s	remaining: 1m 5s
329:	learn: 827.4435734	total: 5.86s	remaining: 1m 5s
330:	learn: 826.9392826	total: 5.87s	remaining: 1m 5s
331:	learn: 826.5722816	total: 5.89s	remaining: 1m 5s
332:	learn: 826.0689646	total: 5.91s	remaining: 1m 5s
333:	learn: 825.7197005	total: 5.93s	remaining: 1m 5s
334:	learn: 825.3202271	total: 5.95s	remaining: 1m 5s
335:	learn: 824.6690890	total: 5.97s	remaining: 1m 5s
336:	learn: 824.3306587	total

471:	learn: 793.0955852	total: 8.23s	remaining: 1m 1s
472:	learn: 793.0197935	total: 8.25s	remaining: 1m 1s
473:	learn: 792.9637057	total: 8.27s	remaining: 1m 1s
474:	learn: 792.8451043	total: 8.28s	remaining: 1m 1s
475:	learn: 792.7673292	total: 8.3s	remaining: 1m 1s
476:	learn: 792.4570504	total: 8.31s	remaining: 1m 1s
477:	learn: 792.3704008	total: 8.33s	remaining: 1m 1s
478:	learn: 792.3182975	total: 8.34s	remaining: 1m 1s
479:	learn: 792.2543309	total: 8.36s	remaining: 1m 1s
480:	learn: 792.1557643	total: 8.37s	remaining: 1m 1s
481:	learn: 792.1080520	total: 8.38s	remaining: 1m 1s
482:	learn: 791.9823060	total: 8.4s	remaining: 1m 1s
483:	learn: 791.8208012	total: 8.41s	remaining: 1m 1s
484:	learn: 791.7554517	total: 8.43s	remaining: 1m 1s
485:	learn: 791.5905154	total: 8.45s	remaining: 1m 1s
486:	learn: 791.5289140	total: 8.46s	remaining: 1m 1s
487:	learn: 791.2763822	total: 8.48s	remaining: 1m 1s
488:	learn: 791.1822113	total: 8.49s	remaining: 1m
489:	learn: 791.1239312	total: 8.

630:	learn: 774.2430373	total: 10.7s	remaining: 57.2s
631:	learn: 774.2145890	total: 10.7s	remaining: 57.2s
632:	learn: 774.1771162	total: 10.7s	remaining: 57.2s
633:	learn: 774.1741493	total: 10.8s	remaining: 57.1s
634:	learn: 774.0812149	total: 10.8s	remaining: 57.1s
635:	learn: 774.0051070	total: 10.8s	remaining: 57s
636:	learn: 773.9152442	total: 10.8s	remaining: 57s
637:	learn: 773.7946828	total: 10.8s	remaining: 57s
638:	learn: 773.6932942	total: 10.8s	remaining: 57s
639:	learn: 773.6001403	total: 10.8s	remaining: 56.9s
640:	learn: 773.5556730	total: 10.9s	remaining: 56.9s
641:	learn: 773.5145052	total: 10.9s	remaining: 56.9s
642:	learn: 773.3008942	total: 10.9s	remaining: 56.8s
643:	learn: 773.2849517	total: 10.9s	remaining: 56.8s
644:	learn: 773.2508874	total: 10.9s	remaining: 56.8s
645:	learn: 773.2160647	total: 10.9s	remaining: 56.8s
646:	learn: 772.9126218	total: 11s	remaining: 56.8s
647:	learn: 772.8762086	total: 11s	remaining: 56.7s
648:	learn: 772.8054992	total: 11s	remai

...

3298:	learn: 697.7104421	total: 53.9s	remaining: 11.5s
3299:	learn: 697.7052646	total: 53.9s	remaining: 11.4s
3300:	learn: 697.7014192	total: 53.9s	remaining: 11.4s
3301:	learn: 697.6927434	total: 54s	remaining: 11.4s
3302:	learn: 697.6792777	total: 54s	remaining: 11.4s
3303:	learn: 697.6658644	total: 54s	remaining: 11.4s
3304:	learn: 697.6468143	total: 54s	remaining: 11.4s
3305:	learn: 697.6464862	total: 54s	remaining: 11.3s
3306:	learn: 697.6464542	total: 54.1s	remaining: 11.3s
3307:	learn: 697.6230162	total: 54.1s	remaining: 11.3s
3308:	learn: 697.6173494	total: 54.1s	remaining: 11.3s
3309:	learn: 697.6125674	total: 54.1s	remaining: 11.3s
3310:	learn: 697.6049545	total: 54.1s	remaining: 11.3s
3311:	learn: 697.6022544	total: 54.2s	remaining: 11.3s
3312:	learn: 697.5966240	total: 54.2s	remaining: 11.2s
3313:	learn: 697.5946066	total: 54.2s	remaining: 11.2s
3314:	learn: 697.5944343	total: 54.2s	remaining: 11.2s
3315:	learn: 697.5776737	total: 54.2s	remaining: 11.2s
3316:	learn: 697.577

3456:	learn: 696.5766873	total: 56.9s	remaining: 8.93s
3457:	learn: 696.5762367	total: 56.9s	remaining: 8.91s
3458:	learn: 696.5742410	total: 56.9s	remaining: 8.9s
3459:	learn: 696.5713784	total: 56.9s	remaining: 8.88s
3460:	learn: 696.5665095	total: 56.9s	remaining: 8.87s
3461:	learn: 696.5570194	total: 57s	remaining: 8.85s
3462:	learn: 696.5525236	total: 57s	remaining: 8.83s
3463:	learn: 696.5518108	total: 57s	remaining: 8.82s
3464:	learn: 696.5416360	total: 57s	remaining: 8.8s
3465:	learn: 696.5398860	total: 57s	remaining: 8.79s
3466:	learn: 696.5261718	total: 57s	remaining: 8.77s
3467:	learn: 696.5175979	total: 57.1s	remaining: 8.75s
3468:	learn: 696.5086946	total: 57.1s	remaining: 8.74s
3469:	learn: 696.5034219	total: 57.1s	remaining: 8.72s
3470:	learn: 696.4941408	total: 57.1s	remaining: 8.7s
3471:	learn: 696.4920419	total: 57.1s	remaining: 8.69s
3472:	learn: 696.4898075	total: 57.1s	remaining: 8.67s
3473:	learn: 696.4828192	total: 57.2s	remaining: 8.65s
3474:	learn: 696.4800614	

3615:	learn: 695.5984940	total: 59.8s	remaining: 6.35s
3616:	learn: 695.5971544	total: 59.8s	remaining: 6.33s
3617:	learn: 695.5930835	total: 59.9s	remaining: 6.32s
3618:	learn: 695.5928532	total: 59.9s	remaining: 6.3s
3619:	learn: 695.5849727	total: 59.9s	remaining: 6.29s
3620:	learn: 695.5760547	total: 59.9s	remaining: 6.27s
3621:	learn: 695.5706113	total: 59.9s	remaining: 6.25s
3622:	learn: 695.5657901	total: 60s	remaining: 6.24s
3623:	learn: 695.5645160	total: 60s	remaining: 6.22s
3624:	learn: 695.5621335	total: 60s	remaining: 6.21s
3625:	learn: 695.5604711	total: 1m	remaining: 6.19s
3626:	learn: 695.5578935	total: 1m	remaining: 6.17s
3627:	learn: 695.5554106	total: 1m	remaining: 6.16s
3628:	learn: 695.5478645	total: 1m	remaining: 6.14s
3629:	learn: 695.5427539	total: 1m	remaining: 6.12s
3630:	learn: 695.5346240	total: 1m	remaining: 6.11s
3631:	learn: 695.5342113	total: 1m	remaining: 6.09s
3632:	learn: 695.5274693	total: 1m	remaining: 6.08s
3633:	learn: 695.5249813	total: 1m	remain

3775:	learn: 694.7319779	total: 1m 2s	remaining: 3.71s
3776:	learn: 694.7300484	total: 1m 2s	remaining: 3.69s
3777:	learn: 694.7253958	total: 1m 2s	remaining: 3.68s
3778:	learn: 694.7238662	total: 1m 2s	remaining: 3.66s
3779:	learn: 694.7205507	total: 1m 2s	remaining: 3.65s
3780:	learn: 694.7064455	total: 1m 2s	remaining: 3.63s
3781:	learn: 694.7061190	total: 1m 2s	remaining: 3.61s
3782:	learn: 694.7055467	total: 1m 2s	remaining: 3.6s
3783:	learn: 694.7002286	total: 1m 2s	remaining: 3.58s
3784:	learn: 694.6885571	total: 1m 2s	remaining: 3.56s
3785:	learn: 694.6872406	total: 1m 2s	remaining: 3.55s
3786:	learn: 694.6854032	total: 1m 2s	remaining: 3.53s
3787:	learn: 694.6782571	total: 1m 2s	remaining: 3.52s
3788:	learn: 694.6746004	total: 1m 2s	remaining: 3.5s
3789:	learn: 694.6723660	total: 1m 2s	remaining: 3.48s
3790:	learn: 694.6595365	total: 1m 2s	remaining: 3.46s
3791:	learn: 694.6530300	total: 1m 2s	remaining: 3.45s
3792:	learn: 694.6509758	total: 1m 2s	remaining: 3.43s
3793:	learn:

3932:	learn: 693.9454357	total: 1m 5s	remaining: 1.11s
3933:	learn: 693.9436601	total: 1m 5s	remaining: 1.1s
3934:	learn: 693.9436122	total: 1m 5s	remaining: 1.08s
3935:	learn: 693.9411683	total: 1m 5s	remaining: 1.06s
3936:	learn: 693.9362554	total: 1m 5s	remaining: 1.05s
3937:	learn: 693.9360343	total: 1m 5s	remaining: 1.03s
3938:	learn: 693.9249344	total: 1m 5s	remaining: 1.01s
3939:	learn: 693.9118550	total: 1m 5s	remaining: 999ms
3940:	learn: 693.9062394	total: 1m 5s	remaining: 983ms
3941:	learn: 693.9053760	total: 1m 5s	remaining: 966ms
3942:	learn: 693.9025157	total: 1m 5s	remaining: 949ms
3943:	learn: 693.8998393	total: 1m 5s	remaining: 933ms
3944:	learn: 693.8997778	total: 1m 5s	remaining: 916ms
3945:	learn: 693.8983361	total: 1m 5s	remaining: 899ms
3946:	learn: 693.8886031	total: 1m 5s	remaining: 883ms
3947:	learn: 693.8818571	total: 1m 5s	remaining: 866ms
3948:	learn: 693.8818330	total: 1m 5s	remaining: 849ms
3949:	learn: 693.8752720	total: 1m 5s	remaining: 833ms
3950:	learn

In [175]:
predicted = cb_model_with_features.predict(test)

In [176]:
predicted

array([ 2579.4910244 ,  3954.13116842,  3202.07048286,  4135.83942371,
        3601.76716427,  3174.46474831,  4074.91453415,  3895.23656684,
        2896.13114111,  3760.43699919,  2514.99379789,  5271.08217397,
        3505.80432965,  3845.51705343,  2155.98249477,  3870.40909073,
        2607.59696783,  2520.95517393,  2410.81348558,  3579.08639327,
        3327.67876244,  2933.16851018,  2454.23668448,  2377.25405744,
        3424.72642716,  3427.89057873,  3081.71063059,  3938.79852295,
        3776.49280294,  3632.32952461,  3698.68353667,  2715.12106255,
        3835.99864165,  3099.55818553,  4800.71853963,  2530.39529138,
        3584.21146004,  4681.71657838,  3008.83078637,  3638.42103859,
        4089.40577051,  3195.14156613,  4088.30319095,  2967.89745302,
        2643.38263715,  3171.97559766,  3835.65546393,  3377.84841043,
        3492.31188638,  4928.11135676,  2856.79701292,  3770.81496627,
        3112.76606195,  2768.17294108,  3340.77443586,  3984.11173058,
      

In [139]:
sub['kda_ratio']=predicted

In [143]:
sub[['id','kda_ratio']].to_csv('solution.csv',index=False)

In [183]:
train

,num_games,primary_attr,attack_type,base_health_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,...,Disabler,Durable,Escape,Initiator,Jungler,Nuker,Pusher,Support,win_hero_percentage,win_user_per
0,304,1,1,1.50,-1.0,25,19,25,16,14,...,1,0,1,0,0,1,1,0,0.56,0.62
1,255,1,1,3.25,0.0,25,14,18,23,15,...,1,0,1,1,1,0,0,0,0.56,0.62
2,203,1,1,1.50,-1.0,25,21,29,16,18,...,0,0,1,0,0,1,0,0,0.54,0.62
3,158,0,0,2.00,-2.0,25,30,34,19,22,...,1,0,1,1,0,1,0,0,0.58,0.62
4,157,2,0,4.25,-1.0,25,24,28,25,20,...,1,1,0,1,1,0,0,0,0.54,0.62
5,154,0,1,1.50,1.0,25,30,36,18,23,...,0,0,1,0,0,0,0,0,0.60,0.62
6,128,2,0,1.50,0.0,25,28,30,26,13,...,1,1,0,1,0,1,0,0,0.57,0.62
7,118,1,1,1.50,2.0,25,22,32,19,22,...,1,0,1,1,0,1,0,0,0.59,0.62
8,116,2,0,1.50,-1.0,25,27,43,26,11,...,1,1,0,1,0,1,0,0,0.56,0.62
9,657,1,1,1.50,-1.0,25,19,25,16,14,...,1,0,1,0,0,1,1,0,0.56,0.54


In [191]:
for c in train.columns:
    if c not in cat_features:
        print(c+"           ",np.std(train[c]))# len(np.unique(train[c])))

num_games            212.550130206
base_health_regen            0.555067834665
base_armor            1.57963414048
base_magic_resistance            0.805819588709
base_attack_min            7.29303513293
base_attack_max            7.52480417431
base_strength            3.33167572539
base_agility            4.32305251082
base_intelligence            4.50791497779
strength_gain            0.530505221855
agility_gain            0.658853041456
intelligence_gain            0.69533551287
attack_range            199.375701209
projectile_speed            437.784455128
attack_rate            0.0961366429501
move_speed            12.2313588945
turn_rate            0.152650029044
win_hero_percentage            0.0227806072919
win_user_per            0.0429684343604


In [187]:
np.unique(train.turn_rate)

array([ 0.5 ,  0.6 ,  0.65,  0.7 ,  0.8 ,  0.9 ,  1.  ])